In [3]:
#pip install datasets

In [4]:
#pip install transformers[torch]

In [5]:
from google.colab import drive
import os
import shutil
import matplotlib.pyplot as plt
import cv2
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, LSTM, TimeDistributed, GRU
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from transformers import AutoImageProcessor, AutoModelForImageClassification, TrainingArguments, Trainer, TrainerCallback
from datasets import load_dataset, Dataset
import os
from PIL import Image

In [6]:
#drive.mount('/content/drive')
import zipfile

test_local_zip = '/content/Gigi Tampak Atas -20240520T120423Z-001.zip'
zip_ref = zipfile.ZipFile(test_local_zip, 'r')
zip_ref.extractall('./')

In [7]:
# Path to the folders containing images
folder_paths = {
    'Gigi Sehat': '/content/Gigi Tampak Atas /Gigi Sehat (Nodup)',
    'Plak Gigi': '/content/Gigi Tampak Atas /Plak Gigi',
    'Gigi Berlubang': '/content/Gigi Tampak Atas /Gigi Berlubang (Nodup)',
    'Bengkak Gusi': '/content/Gigi Tampak Atas /Bengkak Gusi',
    'Bukan Gambar Gigi': '/content/Gigi Tampak Atas /Bukan Gambar Gigi'
}

In [8]:
# Create a directory for the dataset
dataset_dir = '/content/dataset'
os.makedirs(dataset_dir, exist_ok=True)

# Function to copy images to the dataset directory with correct labels
def copy_images_with_labels(folder_paths, dataset_dir):
    for label, folder_path in folder_paths.items():
        image_files = os.listdir(folder_path)
        label_dir = os.path.join(dataset_dir, label)
        os.makedirs(label_dir, exist_ok=True)
        for image_file in image_files:
            shutil.copy2(os.path.join(folder_path, image_file), os.path.join(label_dir, image_file))

# Create dataset
copy_images_with_labels(folder_paths, dataset_dir)

In [ ]:
# Function to display sample images with labels
def display_sample_images(dataset_dir, num_samples=5):
    fig, axes = plt.subplots(len(folder_paths), num_samples, figsize=(15, 10))
    for i, (label, folder_path) in enumerate(folder_paths.items()):
        image_files = os.listdir(os.path.join(dataset_dir, label))
        sampled_images = image_files[:num_samples]
        for j, image_file in enumerate(sampled_images):
            image_path = os.path.join(dataset_dir, label, image_file)
            image = cv2.imread(image_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            axes[i, j].imshow(image)
            axes[i, j].set_title(label)
            axes[i, j].axis('off')
    plt.tight_layout()
    plt.show()

# Display sample images with labels
display_sample_images(dataset_dir)

In [9]:
from transformers import AutoImageProcessor, AutoModelForImageClassification, TrainingArguments, Trainer
from datasets import load_dataset, Dataset, DatasetDict
import torch
import os
from sklearn.model_selection import train_test_split
from PIL import Image

processor = AutoImageProcessor.from_pretrained("Dhyey8/swin-tiny-patch4-window7-224-finetuned-teeth_dataset-finetuned-teeth_dataset-V2")
model = AutoModelForImageClassification.from_pretrained("Dhyey8/swin-tiny-patch4-window7-224-finetuned-teeth_dataset-finetuned-teeth_dataset-V2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/337 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/111M [00:00<?, ?B/s]

In [10]:
def load_images_from_folder(folder):
    images = []
    labels = []
    for label in os.listdir(folder):
        label_path = os.path.join(folder, label)
        if os.path.isdir(label_path):
            for image_file in os.listdir(label_path):
                image_path = os.path.join(label_path, image_file)
                image = Image.open(image_path).convert("RGB")
                images.append(image)
                labels.append(label)
    return images, labels

def create_dataset(dataset_dir):
    images, labels = load_images_from_folder(dataset_dir)
    label_set = list(set(labels))
    label_to_id = {label: i for i, label in enumerate(label_set)}
    id_to_label = {i: label for label, i in label_to_id.items()}

    dataset = Dataset.from_dict({
        "image": images,
        "label": [label_to_id[label] for label in labels]
    })
    return dataset, label_to_id, id_to_label

dataset, label_to_id, id_to_label = create_dataset('/content/dataset')

# Membagi dataset menjadi train dan test
train_test_split = dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [11]:
def preprocess_function(examples):
    inputs = processor(images=examples["image"], return_tensors="pt")
    inputs["label"] = examples["label"]
    return inputs

train_dataset = train_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/2023 [00:00<?, ? examples/s]

Map:   0%|          | 0/506 [00:00<?, ? examples/s]

In [12]:
training_args = TrainingArguments(
    output_dir="./model_tampak_atas",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=15,
    fp16=True,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_total_limit=1,
)

In [13]:
from sklearn.metrics import accuracy_score

# Fungsi compute_metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = accuracy_score(labels, preds)
    return {"accuracy": accuracy}

# Membuat Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=processor,
    compute_metrics=compute_metrics
)

In [14]:
torch.cuda.is_available()

True

In [15]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.326700,0.363814,0.895257
2,0.262900,0.394244,0.881423
3,0.281600,0.312551,0.930830
4,0.191600,0.278782,0.934783
5,0.056300,0.439095,0.916996
6,0.035300,0.312854,0.942688
7,0.035700,0.370315,0.936759
8,0.057800,0.420101,0.922925
9,0.001600,0.389483,0.946640
10,0.000200,0.337173,0.950593


TrainOutput(global_step=1905, training_loss=0.11525047287929148, metrics={'train_runtime': 4502.9419, 'train_samples_per_second': 6.739, 'train_steps_per_second': 0.423, 'total_flos': 7.561517988429619e+17, 'train_loss': 0.11525047287929148, 'epoch': 15.0})

In [16]:
metrics = trainer.evaluate()
print(metrics)

{'eval_loss': 0.3411535322666168, 'eval_accuracy': 0.9565217391304348, 'eval_runtime': 76.7578, 'eval_samples_per_second': 6.592, 'eval_steps_per_second': 0.417, 'epoch': 15.0}


In [17]:
trainer.save_model()
trainer.save_state()

In [19]:
# Evaluasi pada dataset test
metrics = trainer.evaluate()

# Menampilkan hasil evaluasi
print(f"Validation Accuracy: {metrics['eval_accuracy']:.4f}")

# Menampilkan training accuracy dari log yang dihasilkan selama pelatihan
train_metrics = trainer.state.log_history
for epoch_log in train_metrics:
    if "train_accuracy" in epoch_log:
        print(f"Training Accuracy at Epoch {epoch_log['epoch']}: {epoch_log['train_accuracy']:.4f}")

Validation Accuracy: 0.9565


In [40]:
import tensorflow as tf

# Contoh: Mengonversi model dari PyTorch ke TensorFlow
dummy_input = tf.constant(0.0, shape=[1, 224, 224, 3])
tf_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    # Tambahkan lapisan lain sesuai dengan arsitektur model
])
output = tf_model(dummy_input)
tf.saved_model.save(tf_model, "./tf_model")

In [43]:
import tensorflow as tf

# Load the model using tf.saved_model.load
loaded_model = tf.saved_model.load("./tf_model")

# Convert the loaded model to a Keras model
loaded_keras_model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=(224, 224, 3)),  # Add input layer according to your model
    # Add layers from the loaded model
])

# Save the converted model as .h5
loaded_keras_model.save("model_finetuned.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [34]:
# Memuat processor dan model yang sudah dilatih
processor = AutoImageProcessor.from_pretrained('/content/model_tampak_atas')
model = AutoModelForImageClassification.from_pretrained('/content/model_tampak_atas')

# Fungsi untuk memuat dan memproses gambar
def load_and_preprocess_image(image_path, processor):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt")
    return inputs

# Path gambar yang akan diklasifikasikan
image_path = '/content/00000003 (8).jpg'

# Memuat dan memproses gambar
inputs = load_and_preprocess_image(image_path, processor)

# Melakukan inferensi
model.eval()  # Set model ke evaluation mode
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

# Mendapatkan prediksi
predicted_class_idx = logits.argmax(-1).item()

# Mengonversi indeks ke label kelas
id_to_label = {v: k for k, v in label_to_id.items()}  # label_to_id didefinisikan saat membuat dataset
predicted_class_label = id_to_label[predicted_class_idx]

# Menampilkan hasil
print(f"Gambar {image_path} diklasifikasikan sebagai: {predicted_class_label}")

Gambar /content/00000003 (8).jpg diklasifikasikan sebagai: Bengkak Gusi
